In [174]:
import numpy as np
from lxml import html
import requests
import sqlite3
import csv
import pandas as pd
from io import StringIO
import urllib
from bs4 import BeautifulSoup
import re
from datetime import date,timedelta

In [175]:
def MMWRCumtoCurrent(file_name,Sheet_name,Year,header_row,index_column,NaN_value):
    ### converts MMWR cumulative incidence tables to incidence tables  
    df=pd.read_excel(file_name,Sheet_name,header=header_row,index_col=index_column,na_values=NaN_value,thousands=',')
    df=df.replace('--',0)
    df=df.apply(pd.to_numeric,errors='ignore',downcast='float')
    ### Find and interpolate weeks with cum_count=0 whose previous weeks have cum_count >0
    for i in range(1,52):
        df=df.mask((df==0)&(df.shift(i)>0))
    df.interpolate(method='linear', axis=0, limit=None, inplace=True)
    ########
    df2=df.iloc[:,2:].copy()
    df2.drop(df2.index[-1],inplace=True)
    df2.index+=1
    df2.loc[1]=[0.0]*df2.shape[1]
    df2=df2.sort_index(axis=0)
    df.iloc[:,2:]=df.iloc[:,2:].subtract(df2,fill_value=None)
    CurrentWeekTable=df.round()
    #CurrentWeekTable[CurrentWeekTable<0]=0
    return CurrentWeekTable

In [176]:
output_file_name='Pertussis weekly Incidence Data by State 1996-2017(from the cumulative data for the next year table).xlsx'
cum_file_name='Pertussis Cumulative Incidence Data by State 1995-2016 (cumulative data from the next year table).xlsx'
header_row=0
index_column=0
NaN_value=['']



writer = pd.ExcelWriter(output_file_name,engine='xlsxwriter')
#for Year in range(1996,2018): ## This line was used for the data that were extracted from the cum daa of the SAME year table
for Year in range(1995,2017): ## for the data that were extracted from the cum daa of the same NEXT table
    Sheet_name=str(Year)
    print(Year)
    CurrentWeekTable=MMWRCumtoCurrent(cum_file_name,Sheet_name,Year,header_row,index_column,NaN_value)
    CurrentWeekTable.to_excel(writer,Sheet_name)
    
writer.save()               
writer.close()


1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
